In [52]:
from analyze import *
import os

# look through flight_data folder for files containing "aug" (lowercase or uppercase)
# and print out the file names with numbers
files = [f for f in os.listdir("flight_data") if "sep5" in f.lower() or "sepaa" in f.lower()]
for i, f in enumerate(files):
    print(i, f)

0 Sep5_3inch_0percent_randomized_1runs_succesful.csv
1 Sep5_3inch_big_randomization_3runs_succesful.csv
2 Sep5_3inch_30percent_randomized_3runs_succesful.csv
3 Sep5_3inch_20percent_randomized_3runs_succesful.csv
4 Sep5_3inch_0percent_randomized_2runs_succesful_1_optitrack_loss.csv
5 Sep5_3inch_10percent_randomized_3runs_succesful.csv


In [59]:
# select a file by number
num = 3
file = files[num]
print("Selected file:", file)

# load the data
data = load_flight_data("flight_data/" + file, new_format=True)

def trim_nn_active(data):
    indices = data['flightModeFlags'] > 8000
    data = {k: v[indices] for k, v in data.items() if isinstance(v, np.ndarray) and len(v) == len(indices)}
    data['t'] = data['t'] - data['t'][0]
    return data

def trim_time(data, t0=0, tf=12):
    indices = (data['t'] >= t0) & (data['t'] <= tf)
    data = {k: v[indices] for k, v in data.items() if isinstance(v, np.ndarray) and len(v) == len(indices)}
    data['t'] = data['t'] - data['t'][0]
    return data

def split_where_nn_active(data):
    nn_active = data['flightModeFlags'] > 8000
    nn_activate = [i for i in range(1, len(nn_active)) if nn_active[i] and not nn_active[i-1]]
    nn_deactivate = [i for i in range(1, len(nn_active)) if not nn_active[i] and nn_active[i-1]]
    split_data = []
    for i in range(len(nn_activate)):
        split_data.append({k: v[nn_activate[i]:nn_deactivate[i]] for k, v in data.items() if isinstance(v, np.ndarray) and len(v) == len(data['t'])})
        split_data[-1]['t'] = split_data[-1]['t'] - split_data[-1]['t'][0]
    return split_data
    

# trim the data
# data = trim_nn_active(data)

Selected file: Sep5_3inch_20percent_randomized_3runs_succesful.csv
Logging frequency: 872.9291493196605


In [56]:
# animate
# race track
r = 1.5
gate_pos = np.array([
    [ r,  -r, -1.5],
    [ 0,   0, -1.5],
    [-r,   r, -1.5],
    [ 0, 2*r, -1.5],
    [ r,   r, -1.5],
    [ 0,   0, -1.5],
    [-r,  -r, -1.5],
    [ 0,-2*r, -1.5]
])
gate_yaw = np.array([1,2,1,0,-1,-2,-1,0])*np.pi/2

animate_data_multiple(data, gate_pos=gate_pos, gate_yaw=gate_yaw)

In [46]:
# split the data
split_data = split_where_nn_active(data)
# trim time
split_data = [trim_time(d, t0=0, tf=12) for d in split_data]
for i, d in enumerate(split_data):
    print('segment', i, "tf:", d['t'][-1])
animate_data_multiple(*split_data, gate_pos=gate_pos, gate_yaw=gate_yaw)

nn_activate: [17404, 37041, 58531]
nn_deactivate: [27432, 49235, 74293]
segment 0 tf: 11.998643000000001
segment 1 tf: 11.999122999999997
segment 2 tf: 11.998998999999998


In [49]:
%matplotlib
plt.close('all')
# acc_plot(data)
imu_plot(data)
# ekf_plot(data)
# actuator_plot(data)

Using matplotlib backend: TkAgg


In [62]:
# 3inch big randomization
data = load_flight_data("flight_data/Sep5_3inch_big_randomization_3runs_succesful.csv")
runs = split_where_nn_active(data)
runs_3inch_big_domain_randomization = [trim_time(r, t0=0, tf=12) for r in runs]

# 3inch 30 percent randomization
data = load_flight_data("flight_data/Sep5_3inch_30percent_randomized_3runs_succesful.csv")
runs = split_where_nn_active(data)
runs_3inch_30percent_randomization = [trim_time(r, t0=0, tf=12) for r in runs]

Logging frequency: 881.7974382601424


In [63]:
animate_data_multiple(*runs_3inch_30percent_randomization, gate_pos=gate_pos, gate_yaw=gate_yaw, names=['run1', 'run2', 'run3'])

In [6]:
# 0 Sep5_3inch_big_randomization_3runs_succesful.csv
# 1 Sep5_3inch_30percent_randomized_3runs_succesful.csv
# 2 Sep5_3inch_20percent_randomized_3runs_succesful.csv
# 3 Sep5_3inch_10percent_randomized_3runs_succesful.csv

model0 = load_flight_data("flight_data/Sep5_3inch_big_randomization_3runs_succesful.csv")
model1 = load_flight_data("flight_data/Sep5_3inch_30percent_randomized_3runs_succesful.csv")
model2 = load_flight_data("flight_data/Sep5_3inch_20percent_randomized_3runs_succesful.csv")
model3 = load_flight_data("flight_data/Sep5_3inch_10percent_randomized_3runs_succesful.csv")

# trim
model0 = trim_nn_active(model0)
model1 = trim_nn_active(model1)
model2 = trim_nn_active(model2)
model3 = trim_nn_active(model3)

Logging frequency: 906.2990724728469
Logging frequency: 881.7974382601424
Logging frequency: 872.9291493196605
Logging frequency: 902.7468512313426


In [9]:
animate_data_multiple(model0, model1, model2, model3, gate_pos=gate_pos, gate_yaw=gate_yaw, names=['big', '30%', '20%', '10%'])